以下のノートブックを実行するには、まだ行っていない場合、`text-embedding-ada-002` をベースモデルとして使用するモデルをデプロイし、.env ファイル内でデプロイメント名を `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` として設定する必要があります。


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

次に、埋め込みインデックスをPandasデータフレームに読み込みます。埋め込みインデックスは `embedding_index_3m.json` というJSONファイルに保存されています。埋め込みインデックスには、2023年10月下旬までの各YouTubeトランスクリプトの埋め込みが含まれています。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

次に、`get_videos`という関数を作成します。この関数はEmbedding Indexを使ってクエリに合った動画を検索します。関数は、クエリと最も類似している上位5つの動画を返します。関数の動作は以下の通りです。

1. まず、Embedding Indexのコピーを作成します。
2. 次に、OpenAI Embedding APIを使ってクエリのEmbeddingを計算します。
3. その後、Embedding Indexに`similarity`という新しい列を追加します。`similarity`列には、クエリのEmbeddingと各動画セグメントのEmbeddingとのコサイン類似度が入ります。
4. 続いて、Embedding Indexを`similarity`列でフィルタリングします。コサイン類似度が0.75以上の動画だけを残します。
5. 最後に、Embedding Indexを`similarity`列で並べ替え、上位5つの動画を返します。


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

この関数は非常にシンプルで、検索クエリの結果を出力するだけです。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. まず、Embedding Index が Pandas Dataframe に読み込まれます。
2. 次に、ユーザーにクエリの入力が求められます。
3. その後、`get_videos` 関数が呼び出され、Embedding Index からクエリに合ったものを検索します。
4. 最後に、`display_results` 関数が呼び出され、検索結果がユーザーに表示されます。
5. その後、再度クエリの入力が求められます。このプロセスは、ユーザーが `exit` と入力するまで繰り返されます。

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.ja.png)

クエリの入力が求められます。クエリを入力してエンターキーを押してください。アプリケーションは、クエリに関連する動画のリストを返します。また、質問の答えが含まれている動画の該当箇所へのリンクも表示されます。

以下は試してみるクエリの例です：

- Azure Machine Learning とは何ですか？
- 畳み込みニューラルネットワークはどのように動作しますか？
- ニューラルネットワークとは？
- Azure Machine Learning で Jupyter Notebooks を使えますか？
- ONNX とは何ですか？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**免責事項**:  
本書類は、AI翻訳サービス [Co-op Translator](https://github.com/Azure/co-op-translator) を使用して翻訳されています。正確性には努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文（元の言語の文書）が正式な情報源と見なされるべきです。重要な情報については、専門の人間による翻訳を推奨します。本翻訳の利用により生じたいかなる誤解や誤訳についても、当方は一切の責任を負いかねます。
